In [14]:
import pandas as pd
import s3fs
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Especificar la ruta del archivo Parquet en el bucket de S3
s3_path = 's3://agb-prueba-bucket/Covid Data.parquet'

# Leer el archivo Parquet desde S3
covid_data = pd.read_parquet(s3fs.S3FileSystem().open(s3_path))

In [15]:
covid_data.head(5)

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,ASTHMA,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,2,1,2,2,2,2,2,3,97
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,2,1,2,2,1,1,2,5,97
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,2,3,2
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,2,7,97
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,2,1,2,2,2,2,2,3,97


In [16]:
covid_data.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'DATE_DIED', 'INTUBED',
       'PNEUMONIA', 'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU'],
      dtype='object')

In [17]:
covid_data['ICU'].value_counts()

97    848544
2     175685
1      16858
99      7488
Name: ICU, dtype: int64

In [18]:
covid_data["DEATH"] = [1 if each == "9999-99-99" else 0 for each in covid_data.DATE_DIED]
covid_data.head()

,USMER,MEDICAL_UNIT,SEX,PATIENT_TYPE,DATE_DIED,INTUBED,PNEUMONIA,AGE,PREGNANT,DIABETES,...,INMSUPR,HIPERTENSION,OTHER_DISEASE,CARDIOVASCULAR,OBESITY,RENAL_CHRONIC,TOBACCO,CLASIFFICATION_FINAL,ICU,DEATH
0,2,1,1,1,03/05/2020,97,1,65,2,2,...,2,1,2,2,2,2,2,3,97,0
1,2,1,2,1,03/06/2020,97,1,72,97,2,...,2,1,2,2,1,1,2,5,97,0
2,2,1,2,2,09/06/2020,1,2,55,97,1,...,2,2,2,2,2,2,2,3,2,0
3,2,1,1,1,12/06/2020,97,2,53,2,2,...,2,2,2,2,2,2,2,7,97,0
4,2,1,2,1,21/06/2020,97,2,68,97,1,...,2,1,2,2,2,2,2,3,97,0


In [21]:
covid_data = covid_data.drop(labels=["DATE_DIED"],axis=1)

covid_data.columns

Index(['USMER', 'MEDICAL_UNIT', 'SEX', 'PATIENT_TYPE', 'INTUBED', 'PNEUMONIA',
       'AGE', 'PREGNANT', 'DIABETES', 'COPD', 'ASTHMA', 'INMSUPR',
       'HIPERTENSION', 'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY',
       'RENAL_CHRONIC', 'TOBACCO', 'CLASIFFICATION_FINAL', 'ICU', 'DEATH'],
      dtype='object')

In [22]:
X = covid_data.drop(labels=["DEATH"],axis=1)
Y = covid_data["DEATH"]

In [23]:
encoder = LabelEncoder()
Y = encoder.fit_transform(Y)

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [25]:
model = Sequential()
model.add(Dense(32, input_dim=X_train.shape[1], activation='relu'))  # Capa de entrada
model.add(Dense(16, activation='relu'))  # Capa oculta
model.add(Dense(1, activation='sigmoid'))  # Capa de salida

In [26]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, epochs=50, batch_size=32)

Epoch 1/50
26215/26215 [==============================] - 62s 2ms/step - loss: 0.1389 - accuracy: 0.9411
Epoch 2/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1271 - accuracy: 0.9460
Epoch 3/50
26215/26215 [==============================] - 51s 2ms/step - loss: 0.1223 - accuracy: 0.9470
Epoch 4/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1209 - accuracy: 0.9474
Epoch 5/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1200 - accuracy: 0.9478
Epoch 6/50
26215/26215 [==============================] - 51s 2ms/step - loss: 0.1195 - accuracy: 0.9478
Epoch 7/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1191 - accuracy: 0.9479
Epoch 8/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1188 - accuracy: 0.9481
Epoch 9/50
26215/26215 [==============================] - 50s 2ms/step - loss: 0.1186 - accuracy: 0.9481
Epoch 10/50
26215/26215 [==============================

KeyboardInterrupt: 

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Precisión del modelo: {accuracy * 100.0}%')